<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [6]:
!unzip -qq "/content/drive/MyDrive/타이타닉.zip"

In [7]:
!mkdir titanic_pytorch


In [8]:
import shutil
shutil.move('/content/drive/MyDrive/train.csv','/content/drive/MyDrive/titanic_pytorch/train.csv')
shutil.move('/content/drive/MyDrive/test.csv','/content/drive/MyDrive/titanic_pytorch/test.csv')
shutil.move('/content/drive/MyDrive/submission.csv','/content/drive/MyDrive/titanic_pytorch/submission.csv')

'/content/drive/MyDrive/titanic_pytorch/submission.csv'

In [9]:
shutil.move('/content/drive/MyDrive/타이타닉.zip','/content/drive/MyDrive/titanic_pytorch/titanic_data.zip')

'/content/drive/MyDrive/titanic_pytorch/titanic_data.zip'

In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/train.csv')
test = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/submission.csv')

In [5]:
train = train.drop('Ticket',axis = 1)
train = train.drop('Cabin',axis = 1)
test = test.drop('Ticket',axis = 1)
test = test.drop('Cabin',axis = 1)

In [71]:
train = train.fillna({'Embarked':'S'})
test = test.fillna({'Embarked':'S','Fare':20})

In [7]:
em_map = {'S':0,'Q':1,'C':2}

for t in [train] :
  t['Embarked'] = t['Embarked'].map(em_map)
for t in [test] :
  t['Embarked'] = t['Embarked'].map(em_map)

In [8]:
s_map = {'male':1, 'female':2}

for t in [train] :
  t['Sex'] = t['Sex'].map(s_map)
for t in [test] :
  t['Sex'] = t['Sex'].map(s_map)

In [67]:
age_map = {'Mr.':20,'Miss.':15,'Mrs.':20,'Ms.':10,'Mme.':30,'Mlle.':15,'Dona.':30,'Countess.':30,'Sir.':25,'Lady.':20,'Jonkheer.':20,'Don.':20,'Col.':35,'Major.':30,'Capt.':23,'Dr.':30,'Master.':25,'Rev.':35}

for t in [train] :
  for i in range(len(t['Age'])) :
    if np.isnan(t['Age'][i]) :
      for j in age_map :
        if j in t['Name'][i] :
          t['Age'][i] = age_map[j]
for t in [test] :
  for i in range(len(t['Age'])) :
    if np.isnan(t['Age'][i]) :
      for j in age_map :
        if j in t['Name'][i] :
          t['Age'][i] = age_map[j]

<ipython-input-67-69fb9fd690b4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['Age'][i] = age_map[j]


In [73]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

In [74]:
train = train.drop(['PassengerId'], axis=1)
test = test.drop(['PassengerId'], axis=1)

In [77]:
x = train.drop(['Survived'],axis=1)
y = train['Survived']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [80]:
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader

In [81]:
# 정규화? 이거왜함?

scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

scaler.fit(test)
test_scaled = scaler.transform(test)

In [87]:
print(y)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [128]:
train_x = torch.tensor(x_scaled).float()
train_y = torch.tensor(y).resize(891).float()
test_x = torch.tensor(test.values).float()

model = torch.nn.Sequential(
    torch.nn.Linear(7,1),
    torch.nn.Sigmoid()
)

optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)
epochs = 15000

/usr/local/lib/python3.9/dist-packages/torch/_tensor.py:761: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [129]:
for epoch in range(epochs) :
  pred = model(train_x)
  pred = pred.view(-1)
  error = torch.nn.functional.binary_cross_entropy(pred,train_y)

  optimizer.zero_grad()
  error.backward()
  optimizer.step()

  if epoch%10==0 :
    
    output = pred >= torch.FloatTensor([0.5])
    correct = output.float() == train_y
    acc = correct.sum().item()/len(correct)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(epoch,epochs,error.item(),acc*100))

Epoch    0/15000 Cost: 0.744567 Accuracy 42.42%
Epoch   10/15000 Cost: 0.744478 Accuracy 42.42%
Epoch   20/15000 Cost: 0.744388 Accuracy 42.42%
Epoch   30/15000 Cost: 0.744300 Accuracy 42.42%
Epoch   40/15000 Cost: 0.744210 Accuracy 42.42%
Epoch   50/15000 Cost: 0.744121 Accuracy 42.42%
Epoch   60/15000 Cost: 0.744033 Accuracy 42.42%
Epoch   70/15000 Cost: 0.743944 Accuracy 42.31%
Epoch   80/15000 Cost: 0.743855 Accuracy 42.31%
Epoch   90/15000 Cost: 0.743766 Accuracy 42.31%
Epoch  100/15000 Cost: 0.743677 Accuracy 42.31%
Epoch  110/15000 Cost: 0.743588 Accuracy 42.31%
Epoch  120/15000 Cost: 0.743500 Accuracy 42.31%
Epoch  130/15000 Cost: 0.743411 Accuracy 42.31%
Epoch  140/15000 Cost: 0.743322 Accuracy 42.31%
Epoch  150/15000 Cost: 0.743234 Accuracy 42.31%
Epoch  160/15000 Cost: 0.743145 Accuracy 42.31%
Epoch  170/15000 Cost: 0.743057 Accuracy 42.31%
Epoch  180/15000 Cost: 0.742968 Accuracy 42.31%
Epoch  190/15000 Cost: 0.742880 Accuracy 42.31%
Epoch  200/15000 Cost: 0.742791 Accuracy

In [115]:
test_y_pred = model(test_x)

In [124]:
print(test_y_pred)

tensor([[1.8010e-05],
        [1.4119e-07],
        [7.2111e-10],
        [2.5338e-04],
        [3.7253e-03],
        [3.1676e-02],
        [1.3320e-04],
        [3.4807e-03],
        [1.3051e-02],
        [8.0938e-03],
        [3.0494e-03],
        [1.1855e-06],
        [8.2784e-01],
        [2.0698e-09],
        [5.6878e-05],
        [9.8982e-03],
        [2.0834e-05],
        [3.0252e-03],
        [2.5129e-04],
        [6.2680e-07],
        [2.6008e-06],
        [1.2430e-01],
        [4.6520e-02],
        [6.3145e-01],
        [1.0000e+00],
        [7.8704e-08],
        [6.4700e-01],
        [1.7422e-03],
        [1.2758e-05],
        [1.3494e-02],
        [2.4995e-07],
        [5.2195e-03],
        [2.4234e-04],
        [3.9505e-02],
        [2.1125e-02],
        [7.5745e-03],
        [2.7585e-02],
        [3.3262e-03],
        [5.8472e-04],
        [4.8972e-01],
        [1.1618e-05],
        [1.8116e-02],
        [1.3185e-06],
        [1.6440e-04],
        [4.2894e-05],
        [4

In [127]:
sub['Survived'] = torch.Tensor.detach(test_y_pred)
sub.to_csv('베이스 라인 .csv', index = False)